In [ ]:
import logging
import pandas as pd
from google_play_scraper import app
from urllib.error import HTTPError
from datetime import datetime
import os
import schedule
import time

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the app IDs for the banks
app_ids = {
    'Abyssinia Bank': 'com.boa.boaMobileBanking',
    'Commercial Bank of Ethiopia': 'com.combanketh.mobilebanking',
}

def fetch_app_details(app_id):
    """Fetch app details to get download count and other metadata."""
    try:
        details = app(app_id)
        return details
    except HTTPError as e:
        logger.error("HTTPError while fetching details for app ID %s: %s", app_id, e)
        return {}

def track_downloads(app_ids):
    """Track download counts over time."""
    records = []
    
    for bank, app_id in app_ids.items():
        logger.info("Fetching download count for %s", bank)
        
        # Fetch app details
        details = fetch_app_details(app_id)
        if not details:
            logger.info("No app details found for %s", bank)
            continue
        
        download_count = details.get('minInstalls', 0)
        logger.info("Current download count for %s: %d", bank, download_count)
        
        record = {
            'date': datetime.now().strftime('%Y-%m-%d'),
            'bank': bank,
            'app_id': app_id,
            'download_count': download_count
        }
        records.append(record)
    
    # Append to CSV file
    csv_file_path = 'data/google_play_downloads.csv'
    df = pd.DataFrame(records)
    
    if os.path.exists(csv_file_path):
        df_existing = pd.read_csv(csv_file_path)
        df = pd.concat([df_existing, df])
    
    df.to_csv(csv_file_path, index=False)
    logger.info("Saved download counts data to %s", csv_file_path)

def daily_task():
    track_downloads(app_ids)

if __name__ == "__main__":
    # Ensure the data directory exists
    os.makedirs('data', exist_ok=True)
    
    # Schedule the task to run daily at a specific time (e.g., 10:00 AM)
    schedule.every().day.at("10:00").do(daily_task)
    
    logger.info("Scheduled daily task for tracking downloads")
    
    # Run the scheduled task
    while True:
        schedule.run_pending()
        time.sleep(1)